In [1]:
import populator

In [2]:
data_dir = 'data'
# Second item in tuple is max batch size for requests.
exchanges = [('binance', 1000)]
tick_size = '1h'
start = '2000-01-01T00:00:00Z'

In [3]:
stables = ['TUSD/USDT', 'PAX/USDT', 'USDC/USDT', 'USDC/TUSD', 'USDS/TUSD', 'USDC/PAX', 'USDS/USDT']
populator.populate(data_dir, exchanges, stables, tick_size, start)
frames = [(pair, populator.load_data_as_frame(data_dir, 'binance', pair, tick_size, start)) for pair in stables]

Batch Size: 1000
Total Entries: 168335
Note: Entries will not exist if they predate exchange history.
Entries Processed: 162427
Entries Processed: 163427
Entries Processed: 164427
Entries Processed: 165437
Entries Processed: 166437
Entries Processed: 167437
Entries Processed: 168335
Scraping for data/binance/TUSD-USDT-1h-2000-01-01T00-00-00Z-all.csv succeeded.
Batch Size: 1000
Total Entries: 168335
Note: Entries will not exist if they predate exchange history.
Entries Processed: 165444
Entries Processed: 166444
Entries Processed: 167444
Entries Processed: 168335
Scraping for data/binance/PAX-USDT-1h-2000-01-01T00-00-00Z-all.csv succeeded.
Batch Size: 1000
Total Entries: 168335
Note: Entries will not exist if they predate exchange history.
Entries Processed: 167155
Entries Processed: 168155
Entries Processed: 168335
Scraping for data/binance/USDC-USDT-1h-2000-01-01T00-00-00Z-all.csv succeeded.
Batch Size: 1000
Total Entries: 168335
Note: Entries will not exist if they predate exchange h

In [4]:
import backtrader as bt

In [5]:
class StableCoinStrategy(bt.Strategy):
    def next(self):
        if not self.position:
            self.buy(size=self.data.close[0] / (self.data.close[0] + 1.0) * 10000.0)
        elif self.data.close[0] > 1.0:
            self.sell(size=1000)
        else:
            self.buy(size=1000)

In [6]:
def run_strategy(name, df):
    print('Running strategy for {}.'.format(name))
    cerebro = bt.Cerebro()
    cerebro.addstrategy(StableCoinStrategy)
    data = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.00, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

    starting_cash = 10**5
    cerebro.broker.setcash(starting_cash)
    cerebro.broker.setcommission(commission=0.001)
    sim = cerebro.run()
    
    print('Final Value: {}'.format(cerebro.broker.getvalue()))
    print('PNL %: {}'.format(100.0 * (cerebro.broker.getvalue() / starting_cash) - 100.0))
    print('Sharpe Ratio: {}'.format(sim[0].analyzers.sharpe.get_analysis()['sharperatio']))
    print('Max Drawdown: {}'.format(sim[0].analyzers.drawdown.get_analysis().max.drawdown))
    print()

In [7]:
for (name, df) in frames:
    run_strategy(name, df)

Running strategy for TUSD/USDT.
Final Value: 126324.01444609603
PNL %: 26.324014446096015
Sharpe Ratio: 0.6376390028324086
Max Drawdown: 214.4826026548319

Running strategy for PAX/USDT.
Final Value: 127335.05572422873
PNL %: 27.335055724228738
Sharpe Ratio: 0.8380274604865633
Max Drawdown: 51.89555968202013

Running strategy for USDC/USDT.
Final Value: 102027.97844052664
PNL %: 2.0279784405266525
Sharpe Ratio: 1.6281514384982154
Max Drawdown: 7.065335717593181

Running strategy for USDC/TUSD.
Final Value: 99320.84240391752
PNL %: -0.6791575960824758
Sharpe Ratio: None
Max Drawdown: 1.0323255999999528

Running strategy for USDS/TUSD.
Final Value: 99657.1450018013
PNL %: -0.34285499819870324
Sharpe Ratio: None
Max Drawdown: 0.48145219707825165

Running strategy for USDC/PAX.
Final Value: 98783.70780099742
PNL %: -1.2162921990025808
Sharpe Ratio: None
Max Drawdown: 3.4155895180006204

Running strategy for USDS/USDT.
Final Value: 97923.73735308275
PNL %: -2.076262646917243
Sharpe Ratio: N